In [37]:
import numpy as np
import math

all values are in mm (except for the output values - those are in cm)

In [38]:
def arr_str(xs):
    if len(xs.shape) == 1:
        return "[" + ", ".join([f'{x:.6f}' for x in xs]) + "]"
    else:
        res = []
        res.append('[')
        for i in range(xs.shape[0]):
            res.append(arr_str(xs[i]))
            res.append(',\n')
        res.append(']')
        return ''.join(res)

In [39]:
space_scale = 16
obj_scale = 8
n_grid = 64
dx_0 = space_scale / n_grid
min_z = dx_0 * 3
print(f'dx_0: {dx_0}')
print(f'min_z: {min_z}')

phantom_closest_vertex = np.array([min_z+5, min_z+5, min_z], dtype=float)

phantom_h = 2.2
phantom_r = 4
phantom_d = phantom_r * 2
phantom_dimensions = np.array([phantom_d, phantom_d, phantom_h], dtype=float)

phantom_max_dim = np.max(phantom_dimensions)
assert obj_scale == phantom_max_dim
phantom_normalised_spans = phantom_dimensions / phantom_max_dim / 2
phantom_scaled_spans = phantom_normalised_spans * obj_scale

print(f'phantom_scaled_spans: {phantom_scaled_spans}')

phantom_difftactile_position = phantom_closest_vertex + phantom_scaled_spans

sensor_dome_tip = np.array([
    phantom_difftactile_position[0],
    phantom_difftactile_position[1],
    phantom_difftactile_position[2] + phantom_dimensions[2] / 2,
])

print('difftactile coordinates')
print(f'phantom centre of mass: {arr_str(phantom_difftactile_position)}')
print(f'sensor dome tip: {arr_str(sensor_dome_tip)}')



dx_0: 0.25
min_z: 0.75
phantom_scaled_spans: [4.  4.  1.1]
difftactile coordinates
phantom centre of mass: [9.750000, 9.750000, 1.850000]
sensor dome tip: [9.750000, 9.750000, 2.950000]


In [40]:
tumour_cx, tumour_cy, tumour_cz, tumour_theta, tumour_h, tumour_r = (-0.706164183834844, 0.5947180564546317, 0.7677871920541303, 39.402098352330455, 5.097627267173769, 0.22463685189531085)
tumour_volume = math.pi * tumour_r ** 2 * tumour_h
phantom_volume = math.pi * phantom_r ** 2 * phantom_h
fractional_tumour_volume = tumour_volume / phantom_volume
num_total_particles = 23829
num_tumour_particles = num_total_particles * fractional_tumour_volume
print(f'fractional_tumour_volume: {fractional_tumour_volume:.3e}')
print(f'num_tumour_particles: {num_tumour_particles:.2f}')

fractional_tumour_volume: 7.308e-03
num_tumour_particles: 174.14


In [41]:
def global_to_local_velocity(global_velocity, global_transformation_matrix):
    """
    Transforms a 3D velocity vector from the global coordinate frame
    to the local coordinate frame.

    Args:
        global_velocity (np.ndarray): A 3-element numpy array representing
                                       the velocity vector in the global frame (a, b, c).
        global_transformation_matrix (np.ndarray): A 4x4 numpy array representing
                                                   the global transformation matrix.

    Returns:
        np.ndarray: A 3-element numpy array representing the velocity vector
                    in the local coordinate frame.
    """
    if global_velocity.shape != (3,):
        raise ValueError("global_velocity must be a 3-element numpy array.")
    if global_transformation_matrix.shape != (4, 4):
        raise ValueError("global_transformation_matrix must be a 4x4 numpy array.")

    # Extract the 3x3 rotation matrix from the global transformation matrix
    R_global_to_local = global_transformation_matrix[:3, :3]

    # The inverse of a rotation matrix is its transpose
    R_local_to_global = R_global_to_local.T

    # Transform the global velocity vector to the local frame
    local_velocity = np.dot(R_local_to_global, global_velocity)

    return local_velocity

In [42]:
trans_mat = np.array([
    [1, 0, 0, 7.5],
    [0, 0, 1, 6.5],
    [0, -1, 0, 5.4],
    [0, 0, 0, 1],
], dtype=float)

velocity_global = np.array([0, 0, -10], dtype=float)
velocity_local = global_to_local_velocity(velocity_global, trans_mat)
print(velocity_local)

[ 0. 10.  0.]


speed calculator

In [43]:
distance_per_simulation_sub_frame_at_speed_equal_to_1 = 4.898507118225098e-04 # mm
sub_frames_per_frame = 50
frames_per_second = 10
real_world_experiment_duration_seconds = 150
total_num_frames = real_world_experiment_duration_seconds * frames_per_second
total_num_sub_frames = total_num_frames * sub_frames_per_frame
sim_speed_translation = 1 / (distance_per_simulation_sub_frame_at_speed_equal_to_1 * sub_frames_per_frame * frames_per_second) # 1 mm / s

print(sim_speed_translation)

4.0828765820486765


In [44]:
for x, y in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
    a = np.arctan2(x, y)
    b = np.degrees(a)
    print(x, y, '|', a, b)

1 0 | 1.5707963267948966 90.0
0 1 | 0.0 0.0
-1 0 | -1.5707963267948966 -90.0
0 -1 | 3.141592653589793 180.0


In [45]:
angle_1 = 1.063207626735041
angle_n = -2.4003944410946265
d_angle = angle_1 - angle_n
d_time_steps = 900
rot_speed = 90
sub_steps_per_step = 50
angle_rad_per_sub_step_at_speed_1 = d_angle / (d_time_steps * sub_steps_per_step * rot_speed)
print(f'angle_rad_per_sub_step_at_speed_1: {angle_rad_per_sub_step_at_speed_1:e}')

sim_speed_rotation = np.deg2rad(2).item() / (angle_rad_per_sub_step_at_speed_1 * sub_steps_per_step * frames_per_second) # 2 deg / s
print(f'sim_speed_rotation: {sim_speed_rotation:.2f}')

angle_rad_per_sub_step_at_speed_1: 8.552104e-07
sim_speed_rotation: 81.63
